In [1]:
import rasterio
import glob
import uuid
import yaml
import os
from rio_cogeo.cogeo import cog_translate
from rio_cogeo.profiles import cog_profiles


target_aster = 'Gypsum_Index'
product_name = 'aster_'+target_aster.lower()

target_dir = 'ASTER/'+target_aster+'/'

product_definition = {
    'name': product_name,
    'description': 'ASTER',
    'metadata_type':'eo',
    'metadata':{
        'format': {'name':'GeoTIFF'},
        'product_type': product_name
    },
    'measurements':[
        {
            'name':'Band_1',
            'aliases': ['band_01','B01','Band1'],
            'dtype':'uint8',
            'nodata':0,
            'units': '1'
        },
#        {
#            'name':'Band_2',
#            'aliases': ['band_02','B02','Band2'],
#            'dtype':'uint8',
#            'nodata':0,
#            'units': '1'
#        },
#        {
#            'name':'Band_3',
#            'aliases': ['band_03','B03','Band3'],
#            'dtype':'uint8',
#            'nodata':0,
#            'units': '1'
#        }
    ]
    
}

product_filename = target_dir+product_name+'_product.yaml'
with open(product_filename, 'w') as outfile:
    yaml.dump(product_definition, outfile, default_flow_style=False)
    print('created product document '+product_filename)

dataset_constants = {
    'creation_dt':'2012-01-01',
    'extent':{
        'center_dt':'2004-03-01 00:00:01.0000000Z',
        'from_dt':'2000-02-01 00:00:01.0000000Z',
        'to_dt':'2008-03-31 23:59:59.9999999Z',
        },
    'product_type':product_name,
    'format':{'name':'GeoTIFF'},
    'lineage':{'source_datasets': {}}
}

created product document ASTER/Gypsum_Index/aster_gypsum_index_product.yaml


In [2]:
def create_cog(input, output, bidx):
    cogeo_profile = 'deflate'
    nodata = None
    alpha = None
    overview_level = 6
    overview_resampling = 'average'
    threads = 8
    
    output_profile = cog_profiles.get(cogeo_profile)
    output_profile.update(dict(BIGTIFF=os.environ.get("BIGTIFF", "IF_SAFER")))
    
    block_size = min(
        int(output_profile["blockxsize"]), int(output_profile["blockysize"])
    )

    config = dict(
        NUM_THREADS=threads,
        GDAL_TIFF_INTERNAL_MASK=os.environ.get("GDAL_TIFF_INTERNAL_MASK", True),
        GDAL_TIFF_OVR_BLOCKSIZE=os.environ.get("GDAL_TIFF_OVR_BLOCKSIZE", block_size),
    )
    print('creating '+output)
    cog_translate(
        input,
        output,
        output_profile,
        bidx,
        nodata,
        alpha,
        overview_level,
        overview_resampling,
        config
    )
    print('created '+output)

In [3]:
index_commands = []
tifs = glob.glob(target_dir+'*.tif', recursive=True)

In [4]:
for tif in tifs:
    with rasterio.open(tif, 'r') as img:
        print('processing '+tif)
        dataset_doc = dataset_constants.copy()
        dataset_doc['grid_spatial'] = { 'projection':{'spatial_reference':str(img.read_crs())}}
        dataset_doc['id'] = str(uuid.uuid4())
        left = img.bounds.left
        bottom = img.bounds.bottom
        right = img.bounds.right
        top = img.bounds.top

        dataset_doc['extent']['coord'] = {
            'll':{'lat':bottom,'lon':left},
            'lr':{'lat':bottom,'lon':right},
            'ul':{'lat':top,'lon':left},
            'ur':{'lat':top,'lon':right}
        }
        dataset_doc['grid_spatial']['projection']['geo_ref_points'] = {
            'll':{'y':bottom,'x':left},
            'lr':{'y':bottom,'x':right},
            'ul':{'y':top,'x':left},
            'ur':{'y':top,'x':right}
        }
        
        dataset_doc['image'] = {'bands':{}}
        
        for index in img.indexes:
            cog_name = tif.replace('.tif','_Band'+str(index)+'.tif')
            create_cog(tif, cog_name, [index])
            dataset_doc['image']['bands']['Band_'+str(index)] = {'layer':'1', 'path':os.path.basename(cog_name)}
        
        yaml_filename = tif.replace('.tif','.yaml')
        with open(yaml_filename, 'w') as outfile:
            yaml.dump(dataset_doc, outfile, default_flow_style=False)
            print('created dataset document '+yaml_filename)
        index_commands.append('datacube dataset add '+os.path.basename(yaml_filename)+'\n')
    os.remove(tif)
    print('deleted '+tif)

index_filename = target_dir + 'index_datasets.sh'
with open(index_filename, 'w') as outfile:
    outfile.write('datacube product add '+os.path.basename(product_filename) +'\n')
    for index_command in index_commands:
        outfile.write(index_command)
    print('created indexing script '+index_filename)


processing ASTER/Gypsum_Index/SH55_Gypsum_Index.tif
creating ASTER/Gypsum_Index/SH55_Gypsum_Index_Band1.tif
created ASTER/Gypsum_Index/SH55_Gypsum_Index_Band1.tif
created dataset document ASTER/Gypsum_Index/SH55_Gypsum_Index.yaml
deleted ASTER/Gypsum_Index/SH55_Gypsum_Index.tif
processing ASTER/Gypsum_Index/SG53_Gypsum_Index.tif
creating ASTER/Gypsum_Index/SG53_Gypsum_Index_Band1.tif
created ASTER/Gypsum_Index/SG53_Gypsum_Index_Band1.tif
created dataset document ASTER/Gypsum_Index/SG53_Gypsum_Index.yaml
deleted ASTER/Gypsum_Index/SG53_Gypsum_Index.tif
processing ASTER/Gypsum_Index/SF52_Gypsum_Index.tif
creating ASTER/Gypsum_Index/SF52_Gypsum_Index_Band1.tif
created ASTER/Gypsum_Index/SF52_Gypsum_Index_Band1.tif
created dataset document ASTER/Gypsum_Index/SF52_Gypsum_Index.yaml
deleted ASTER/Gypsum_Index/SF52_Gypsum_Index.tif
processing ASTER/Gypsum_Index/SG50_Gypsum_Index.tif
creating ASTER/Gypsum_Index/SG50_Gypsum_Index_Band1.tif
created ASTER/Gypsum_Index/SG50_Gypsum_Index_Band1.tif


created ASTER/Gypsum_Index/SG55_Gypsum_Index_Band1.tif
created dataset document ASTER/Gypsum_Index/SG55_Gypsum_Index.yaml
deleted ASTER/Gypsum_Index/SG55_Gypsum_Index.tif
processing ASTER/Gypsum_Index/SH56_Gypsum_Index.tif
creating ASTER/Gypsum_Index/SH56_Gypsum_Index_Band1.tif
created ASTER/Gypsum_Index/SH56_Gypsum_Index_Band1.tif
created dataset document ASTER/Gypsum_Index/SH56_Gypsum_Index.yaml
deleted ASTER/Gypsum_Index/SH56_Gypsum_Index.tif
processing ASTER/Gypsum_Index/SH50_Gypsum_Index.tif
creating ASTER/Gypsum_Index/SH50_Gypsum_Index_Band1.tif
created ASTER/Gypsum_Index/SH50_Gypsum_Index_Band1.tif
created dataset document ASTER/Gypsum_Index/SH50_Gypsum_Index.yaml
deleted ASTER/Gypsum_Index/SH50_Gypsum_Index.tif
processing ASTER/Gypsum_Index/SH52_Gypsum_Index.tif
creating ASTER/Gypsum_Index/SH52_Gypsum_Index_Band1.tif
created ASTER/Gypsum_Index/SH52_Gypsum_Index_Band1.tif
created dataset document ASTER/Gypsum_Index/SH52_Gypsum_Index.yaml
deleted ASTER/Gypsum_Index/SH52_Gypsum_In